In [16]:
import json
import cv2
import numpy as np
from skimage.morphology import skeletonize
from scipy.ndimage import distance_transform_edt

roi_path  = r"roi_stenosis.png"
mask_path = r"mmask.png"
meta_path = r"roi_stenosis.json"

roi  = cv2.imread(roi_path)
mask = cv2.imread(mask_path, 0)

if roi is None:
    raise FileNotFoundError(roi_path)
if mask is None:
    raise FileNotFoundError(mask_path)

with open(meta_path) as f:
    meta = json.load(f)

sx1, sy1, sx2, sy2 = meta["yolo_box"]


In [17]:
binary = mask > 0
skeleton = skeletonize(binary)
dist = distance_transform_edt(binary)

coords = np.column_stack(np.where(skeleton))  # (y, x)
diameters = dist[skeleton] * 2


In [18]:
inside = (
    (coords[:,1] >= sx1) & (coords[:,1] <= sx2) &
    (coords[:,0] >= sy1) & (coords[:,0] <= sy2)
)

outside = ~inside


In [19]:
diam_inside = diameters[inside]
diam_outside = diameters[outside]

if len(diam_inside) < 5 or len(diam_outside) < 10:
    raise ValueError("Invalid geometry — insufficient points")

D_min = np.min(diam_inside)
D_ref = np.mean(np.sort(diam_outside)[-10:])

stenosis = (1 - D_min / D_ref) * 100

print(f"D_min = {D_min:.2f} px")
print(f"D_ref = {D_ref:.2f} px")
print(f"Stenosis = {stenosis:.1f}%")


D_min = 5.66 px
D_ref = 10.33 px
Stenosis = 45.2%


In [9]:
inside = (
    (coords[:, 1] >= sx1) & (coords[:, 1] <= sx2) &
    (coords[:, 0] >= sy1) & (coords[:, 0] <= sy2)
)

outside = ~inside


In [20]:
diam_inside = diameters[inside]
diam_outside = diameters[outside]

if len(diam_inside) < 1 or len(diam_outside) < 10:
    raise ValueError("Invalid geometry — insufficient points")

D_min = np.min(diam_inside)
D_ref = np.mean(np.sort(diam_outside)[-10:])

stenosis = (1 - D_min / D_ref) * 100

print(f"D_min = {D_min:.2f} px")
print(f"D_ref = {D_ref:.2f} px")
print(f"Stenosis = {stenosis:.1f}%")


D_min = 5.66 px
D_ref = 10.33 px
Stenosis = 45.2%


In [21]:
vis = roi.copy()

# YOLO stenosis region
cv2.rectangle(vis, (sx1, sy1), (sx2, sy2), (255, 0, 0), 2)

# D_min (red)
idx_min = np.where(inside)[0][np.argmin(diam_inside)]
y_min, x_min = coords[idx_min]
cv2.circle(vis, (x_min, y_min), 4, (0, 0, 255), -1)

# D_ref (green)
idx_ref = np.where(outside)[0][np.argmax(diam_outside)]
y_ref, x_ref = coords[idx_ref]
cv2.circle(vis, (x_ref, y_ref), 4, (0, 255, 0), -1)

cv2.putText(
    vis,
    f"Stenosis: {stenosis:.1f}%",
    (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX,
    0.8,
    (0, 255, 255),
    2
)

cv2.imshow("YOLO-guided Stenosis Quantification", vis)
cv2.waitKey(0)
cv2.destroyAllWindows()
